Setup: These commands need to be run before using our program.

In [ ]:
!pip install pytorch_lightning
!pip install torchsummaryX
!pip install webdataset
!git clone --branch master https://github.com/McMasterAI/Radiology-and-AI.git  
!git clone https://github.com/black0017/MedicalZooPytorch.git

We can get set-up with Google Colab if were using it.

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
cd drive/MyDrive/MacAI

Imports

In [ ]:
import sys
sys.path.append('./Radiology-and-AI/Radiology_and_AI')
sys.path.append('./MedicalZooPytorch')
import os
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader, random_split
from pytorch_lightning.loggers import WandbLogger, TensorBoardLogger
import pytorch_lightning as pl
import sys
import nibabel as nb
from skimage import transform
import matplotlib.pyplot as plt
import webdataset as wds
from collators.brats_collator import col_img
from lightning_modules.segmentation import TumourSegmentation
from scipy.interpolate import RegularGridInterpolator
from scipy.ndimage.filters import gaussian_filter
from time import time

Loading datasets. 
Because neuroimages are really large files, we've decided to use the webdataset library to handle them during training. Essentially, we create a zip file representing our dataset and store them in some file path. However, we can work with any PyTorch dataset object (check PyTorch dataset documentation for details).

In [ ]:
train_dataset = wds.Dataset("macai_datasets/brats/train/brats_train.tar.gz")
eval_dataset = wds.Dataset("macai_datasets/brats/validation/brats_validation.tar.gz")

To modify/load in the dataset, we use a *collator function* which is also imported (called col_img). You should create a lambda function only taking in the DataLoader batch as an argument, and using whatever arguments you want afterwards. This sounds complex, so just check the next examples:

A few notes:
- Image augmentations randomly change training images, to artificially increase the sample size by a bit. The available augmentations, demonstrated to be most effective in literature, are the power-law transformation and elastic transformation. However, elastic transformation is relatively slow as of now. Set the augmentation probabilities (pl_prob and elastic_prob) to 0 during evaluation, but you can set them between 0 and 1 for training.
- Image normalization is used to make the image intensity distributions more similar. We currently support two types: Nyul normalization and Z-score normalization. To use Z-score normalization, set use_zscore to True. To use Nyul normalization, the *standard_scales* and *percs* have to be trained first (more details later)

Note: both Nyul normalization and Z-score normalization will normalize based on the non-background (black) pixels of the entire image, including the tumor region.

In [ ]:
training_collator_function = lambda batch: col_img(batch, to_tensor=True, nyul_params=None, use_zscore=True, pl_prob=0.5, elastic_prob=0)

In [ ]:
eval_collator_function = lambda batch: col_img(batch, to_tensor=True, nyul_params=None, use_zscore=True, pl_prob=0, elastic_prob=0)

Nyul normalization can be trained using the training dataset. We first create a dataloader that uses a collator function that makes no changes to the image, then feed it to an imported nyul_train_dataloader function. While this currently ignores the segmented region and background (for more accurate use in radiomics), we will create an option to also take into account the segmented region (as we won't have access to a segmentation before performing automated segmentation).

In [ ]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=5, collate_fn=lambda batch:col_img(batch, to_tensor=False))
standard_scales, percss = nyul_train_dataloader(train_dataloader, step = 5)

After training, we can apply Nyul normalization to our dataset.

In [ ]:
nyul_collator_function = lambda batch: col_img(batch, to_tensor=True, nyul_params=nyul_params={'percs': percss, 'standard_scales':standard_scales})

We have a lot going on in this line.
Our model is a PyTorch Lightning model called TumourSegmentation, which we import above. This instantiates a new instance of the model, and i used during training.
- The learning rate controls how quickly the model learns. Too high, and the model won't converge; too low, and it will take too long to train.
- The collator is described previously.
- The train_dataset is what we train the model using, and the eval_dataset is to ensure that our model is truly learning (rather than memorizing the train_dataset).
- batch_size has to be set to the number of images in each series (including the segmentation image). In this case, we have 4 (T1, T2, T1ce, T1 FLAIR) plus a segmentation, to make a total of 5.


In [ ]:
model = TumourSegmentation(learning_rate = 4e-4, collator=collator_function, batch_size=5, train_dataset=train_dataset, eval_dataset=eval_dataset)

This code deals with training. We can check tensorboard to see how well it's been running after training; you can also use any other type of logger. I use tensorboard here, but there exists another (WandB) that handles automatic updating on Colab.

In [ ]:
%load_ext tensorboard

In [ ]:
#Training
#wandb_logger = WandbLogger(project='macai',name='test_run', offline = True)
trainer = pl.Trainer(
    accumulate_grad_batches = 1,
    gpus = 1,
    max_epochs = 10,
    precision=16,
    check_val_every_n_epoch = 1,
    logger = tensorboard_logger,
    log_every_n_steps=10,            
)
trainer.fit(model)

In [ ]:
%tensorboard --logdir logs/

The trainer automatically creates checkpoints, but we can interrupt the trainer and save a checkpoint like so whenever we wish.

In [ ]:
trainer.save_checkpoint("last_ckpt.ckpt")

Finally, it is possible to load saved models and to see the outputs. We can either visualize this in a Python notebook, or by saving the segmentation somewhere and visualizing it using a neuroimaging software (I use 3D Slicer, but I think anything will do).

In [ ]:
# Load the model
model = TumourSegmentation.load_from_checkpoint('last_ckpt.ckpt').cuda().half()
i=0

for z in model.val_dataloader():
  print('======================================================')
  prediction = model.forward(torch.unsqueeze(z[0], axis=0).cuda().half())

  # Save predictions to file for further visualization
  prediction_img = nb.Nifti1Image(prediction, np.eye(4))
  nb.save(prediction_img, 'prediction_'+str(i)+'.nii.gz')

  # Simple visualization of a slice, but we can use Cameron's visualization method
  # for improvements to this process.

  sl = z[1][0, :, 100]

  plt.title('Label')
  plt.imshow(sl, vmin = 0, vmax=4)
  plt.show()

  prediction = prediction[0].cpu().detach().numpy().astype('float32')

  plt.title('Prediction core')
  plt.imshow(prediction[0, :, 100], vmin = 0, vmax=1)
  plt.show()

  plt.title('Prediction enhancing')
  plt.imshow(prediction[1, :, 100], vmin = 0, vmax=1)
  plt.show()

  plt.title('Prediction edema')
  plt.imshow(prediction[2, :, 100], vmin = 0, vmax=1)
  plt.show()

  i += 1
  if i >= 10:
    break